In [42]:
from trajectory_utils import filter_joint_states
import json

input_file = "split_lego_both.json"  # Update the path as needed
with open(input_file, "r") as file:
    js = json.load(file)

In [45]:
filtered_js = filter_joint_states(js, 0.01)
print(len(filtered_js))

for i, _ in enumerate(filtered_js):
    if i < 100:
        js[i]['gripper_state'] = ('open_left', 'open_right')
    else:
        js[i]['gripper_state'] = ('close_left', 'open_right')

with open("split_lego_both.json", "w") as file:
    json.dump(filtered_js, file, indent=4)

211


In [40]:
len(filtered_js)

211

In [1]:
from trajectory_utils import filter_joint_states
import json

input_file = "split_lego_both.json"  # Update the path as needed

with open(input_file, "r") as file:
    js = json.load(file)
    
filtered_joint_states = filter_joint_states(js, 0.001)
len(filtered_joint_states)

211

In [2]:
from ikSolver import IKSolver
from get_fk import GetFK
from trajectory_utils import filter_joint_states
from replay_trajectory import dict_to_joint_state

gfk_left = GetFK('gripper_l_base', 'world')
gfk_right = GetFK('gripper_r_base', 'world')

msgs = [dict_to_joint_state(filtered_joint_state) for filtered_joint_state in filtered_joint_states]

eef_poses_left = [gfk_left.get_fk(msg) for msg in msgs]
eef_poses_right = [gfk_right.get_fk(msg) for msg in msgs]

In [3]:
left_eef = "gripper_l_base"
right_eef = "gripper_r_base"

ik_solver_left = IKSolver(group_name="left_arm", ik_link_name=left_eef)
ik_solver_right = IKSolver(group_name="right_arm", ik_link_name=right_eef)

waypoints_left = [eef_pose.pose_stamped[0].pose for eef_pose in eef_poses_left]
waypoints_right = [eef_pose.pose_stamped[0].pose for eef_pose in eef_poses_right]
# Example coordinates and orientation (Euler angles in radians)
# for eef_pose_left in waypoints_left:
#     ik_left = ik_solver_left.get_ik(eef_pose_left).solution.joint_state.position


[ WARN] [1706900573.268136034]: IK plugin for group 'left_arm' relies on deprecated API. Please implement initialize(RobotModel, ...).
[ WARN] [1706900573.273064414]: IK plugin for group 'right_arm' relies on deprecated API. Please implement initialize(RobotModel, ...).


[ INFO] [1706900573.139605928]: Loading robot model 'yumi'...
[ INFO] [1706900573.270880324]: IK Using joint yumi_link_1_l -2.94088 2.94088
[ INFO] [1706900573.270899531]: IK Using joint yumi_link_2_l -2.50455 0.759218
[ INFO] [1706900573.270910775]: IK Using joint yumi_link_3_l -2.94088 2.94088
[ INFO] [1706900573.270921670]: IK Using joint yumi_link_4_l -2.15548 1.39626
[ INFO] [1706900573.270930261]: IK Using joint yumi_link_5_l -5.06145 5.06145
[ INFO] [1706900573.270940109]: IK Using joint yumi_link_6_l -1.53589 2.40855
[ INFO] [1706900573.270949537]: IK Using joint yumi_link_7_l -3.9968 3.9968
[ INFO] [1706900573.270963506]: Looking in common namespaces for param name: left_arm/position_only_ik
[ INFO] [1706900573.272039417]: Looking in common namespaces for param name: left_arm/solve_type
[ INFO] [1706900573.273026560]: Using solve type Manipulation2
[ INFO] [1706900573.275314853]: IK Using joint yumi_link_1_r -2.94088 2.94088
[ INFO] [1706900573.275328612]: IK Using joint yumi_

[ WARN] [1706900573.420716480]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420759852]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420782829]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420808461]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420842404]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900573.419327 according t

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def solve_ik(ik_solver, pose):
    return ik_solver.get_ik(pose).solution.joint_state.position

# Using ThreadPoolExecutor to parallelize IK solving
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
    futures_left = {executor.submit(solve_ik, ik_solver_left, pose): pose for pose in waypoints_left}
    futures_right = {executor.submit(solve_ik, ik_solver_right, pose): pose for pose in waypoints_right}

    results_left = [future.result() for future in as_completed(futures_left)]
    results_right = [future.result() for future in as_completed(futures_right)]


In [1]:
import rospy
import moveit_commander
import yumi_moveit_utils as yumi
import moveit_msgs.msg
import geometry_msgs.msg
from std_srvs.srv import Empty
import json
import tf
import matplotlib.pyplot as plt
import copy
from get_fk import GetFK
from sensor_msgs.msg import JointState
from trajectory_utils import filter_joint_states, translation_from_matrix, quaternion_from_matrix
import numpy as np
import tf.transformations as tf_trans
from scipy.spatial.transform import Rotation as R
from replay_trajectory import dict_to_joint_state
import threading
from ikSolver import IKSolver
from trajectory_utils import filter_joint_states, apply_transformation_to_waypoints
from replay_trajectory import dict_to_joint_state, display_trajectory, move_upwards
from poseEstimation import PoseEstimation

def load_grasp_pose():

    gfk_left = GetFK('gripper_l_base', 'world')
    file_name="data/split_lego/left.json"

    with open(file_name) as f:
        joint_states = json.load(f)

    msg = dict_to_joint_state(joint_states[0])

    return gfk_left.get_fk(msg).pose_stamped[0].pose 

waypoints = [load_grasp_pose()]
waypoints_np = np.array([[waypoint.position.x, waypoint.position.y, waypoint.position.z,
                        waypoint.orientation.x, waypoint.orientation.y, waypoint.orientation.z,
                        waypoint.orientation.w] for waypoint in waypoints])

transformed_fine_waypoints = apply_transformation_to_waypoints(waypoints_np, np.eye(4))
fine_waypoints = [yumi.create_pose(*waypoint) for waypoint in transformed_fine_waypoints]

print(fine_waypoints)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[position: 
  x: 0.37569364072211847
  y: 0.15604071247386805
  z: 0.24954663886436484
orientation: 
  x: 0.9324422136326348
  y: -0.02422534821019132
  z: -0.02523011327691357
  w: 0.3596221518816671]


In [2]:
from get_fk import GetFK
import json
from replay_trajectory import dict_to_joint_state

def load_grasp_pose():

    gfk_left = GetFK('gripper_l_base', 'world')
    file_name="data/split_lego/left.json"

    with open(file_name) as f:
        joint_states = json.load(f)

    msg = dict_to_joint_state(joint_states[0])

    return gfk_left.get_fk(msg).pose_stamped[0].pose 

bottle_neck_pose = load_grasp_pose()
bottle_neck_pose

position: 
  x: 0.37569364072211847
  y: 0.15604071247386805
  z: 0.24954663886436484
orientation: 
  x: 0.9324422136326349
  y: -0.024225348210191323
  z: -0.025230113276913574
  w: 0.35962215188166713